This converts `fingerprints.npy` to `.tsv` formatted t-SNE embeddings and plots of those embeddings in the `tsne/` and `plot/` folders respectively. If you add multiple values to `perplexity` and `initial_dims` then all combinations will be computed (in parallel). Good perplexities are in the range 1-200 with the best range around 30-100. Good `initial_dims` are in the range 30 and higher, with the dimensionality of your input data being the highest possible value (e.g., a 32x32 fingerprint would have a highest possible `initial_dims` value of 32x32=1024).

Change the "mode" to try different t-SNE variations.
* "fingerprints" will only use `fingerprints.npy`
* "predicted_labels" will only use `predicted_labels.npy`
* "predicted_encoding" will only use `predicted_encoding.npy`
* "combined" will use all of the above data

In [1]:
data_root = 'drumData/'
initial_dims = [30]
perplexities = [30]
mode = 'fingerprints'
drumNames = ["kick", "tom", "snare", "clap", "hi.hat", "ride", "crash"] 
drumLengths = [] #number of items in ith class
colors = ['#000000', '#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff', '#00ffff']
# mode = 'predicted_labels'
# mode = 'predicted_encoding'
# mode = 'combined'

In [40]:
%matplotlib inline
from matplotlib import pyplot as plt
from time import time
from utils import *
from os.path import join
from multiprocessing import Pool
import numpy as np
import itertools
import time as timeMod
from time import mktime
from datetime import datetime
import scipy.spatial

In [3]:
X_2d_inspect = None

def save_tsv(data, fn):
    np.savetxt(fn, data, fmt='%.5f', delimiter='\t')
def tsne(data, data_root, prefix, colorMap, initial_dims=30, perplexity=30):
    mkdir_p(data_root + 'tsne')
    mkdir_p(data_root + 'plot')
    
    figsize = (16,16)
    pointsize = 30
    
    struct = timeMod.localtime(time())
    dt = datetime.fromtimestamp(mktime(struct))

    X_2d = list(bh_tsne(data, initial_dims=initial_dims, perplexity=perplexity, no_dims=2))
    X_2d = normalize(np.array(X_2d))
    save_tsv(X_2d, join(data_root, 'tsne/{}.{}.{}.2d - {}.tsv'.format(prefix, initial_dims, perplexity, dt)))
    
    X_2d_inspect = X_2d
    
    plt.figure(figsize=figsize)
    plt.scatter(X_2d[:,0], X_2d[:,1], c=colorMap, s=pointsize)
    plt.tight_layout()
    plt.savefig(join(data_root, 'plot/{}.{}.{}_2D - {}.png'.format(prefix, initial_dims, perplexity, dt)))
    plt.close()
    
    return X_2d
#     X_3d = list(bh_tsne(data, initial_dims=initial_dims, perplexity=perplexity, no_dims=3))
#     X_3d = normalize(np.array(X_3d))
#     save_tsv(X_3d, join(data_root, 'tsne/{}.{}.{}.3d.tsv'.format(prefix, initial_dims, perplexity)))
    
#     plt.figure(figsize=figsize)
#     plt.scatter(X_2d[:,0], X_2d[:,1], edgecolor='', s=pointsize, c=X_3d)
#     plt.tight_layout()
#     plt.savefig(join(data_root, 'plot/{}.{}.{}_3D.png'.format(prefix, initial_dims, perplexity)))
#     plt.close()
    
def concatColors(segmentList, colorList):
    multiples = []
    print segmentList, colorList
    for i in range(len(segmentList)):
        multiples.append([colorList[i]]*segmentList[i])
    return list(itertools.chain(*multiples))

In [4]:
# load and normalize any dataset we need
if mode == 'fingerprints' or mode == 'combined':
    drumPrints = []
    for drum in drumNames:
        drumPrints.append(np.load(join(data_root, drum+'_fingerprints.npy')))
        print drum, drumPrints[-1].shape
    drumLengths = [drumPrint.shape[0] for drumPrint in drumPrints]
    colorMap = concatColors(drumLengths, colors)
    fingerprints = np.vstack(drumPrints)
    fingerprints = fingerprints.reshape(len(fingerprints), -1)
if mode == 'predicted_labels' or mode == 'combined':
    predicted_labels = np.load(join(data_root, 'predicted_labels.npy'))
    predicted_labels -= predicted_labels.min()
    predicted_labels /= predicted_labels.max()
if mode == 'predicted_encoding' or mode == 'combined':
    predicted_encoding = np.load(join(data_root, 'predicted_encoding.npy'))
    std = predicted_encoding.std(axis=0)
    predicted_encoding = predicted_encoding[:, std > 0] / std[std > 0]
    
if mode == 'fingerprints':
    data = fingerprints
if mode == 'predicted_labels':
    data = predicted_labels
if mode == 'predicted_encoding':
    data = predicted_encoding
if mode == 'combined':
    data = np.hstack((fingerprints, predicted_labels, predicted_encoding))

print "generating 1024 dim KD tree, this will take a couple minutes"
kdTree1024Dim = scipy.spatial.KDTree(data)    


kick (5158, 32, 32)
tom (422, 32, 32)
snare (2546, 32, 32)
clap (1324, 32, 32)
hi.hat (159, 32, 32)
ride (228, 32, 32)
crash (723, 32, 32)
[5158, 422, 2546, 1324, 159, 228, 723] ['#000000', '#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff', '#00ffff']
(10560, 1024)


In [124]:
data = data.astype(np.float64)
def job(params):
    start = time()
    data2d = tsne(data, data_root, mode, colorMap, initial_dims=params[0], perplexity=params[1])
    print 'initial_dims={}, perplexity={}, {} seconds'.format(params[0], params[1], time() - start)
    return data2d
params = list(itertools.product(initial_dims, perplexities))
pool = Pool()
dimReducedArrays = pool.map(job, params)
newData = dimReducedArrays[0]
kdTree2Dim = scipy.spatial.KDTree(newData)

initial_dims=30, perplexity=30, 124.192649126 seconds


In [123]:
print dimReducedArrays

 [None]


In [115]:


def getClassesPerSample(data):
    drumClasses = {}
    classIndex = 0
    for i in range(len(data)):
        if sum(drumLengths[0:classIndex+1]) <= i:
            classIndex += 1
        drumClasses[tuple(data[i])] = drumNames[classIndex]
    return drumClasses

drumClasses = getClassesPerSample(data)


def calculateKNNClassAccuracy(data, numNeighbors, printTiming=False, existingTree=None):
    startTime = 0
    if printTiming:
        print "building tree", timeMod.time()-startTime
    
    if existingTree is None:
        kdTree = scipy.spatial.KDTree(data) #takes about 2 minutes for 1024 dims
    else:
        kdTree = existingTree
    
    if printTiming:
        print "tree built", timeMod.time()-startTime 
        
    drumClasses = getClassesPerSample(data)
    
    return calcNeighborDistances(data, numNeighbors, kdTree, True)

        
def calcNeighborDistances(pointData, numNeighbors, kdTree, printTiming=False):
    startTime = timeMod.time()
    if printTiming:
        print "calculating nearest neighbor classes"
    fracCorrectNeighbors = np.zeros((len(data)))
    for i in range(len(pointData)):
        if printTiming:
            if i % 1000 == 0:
                print "evaluating neighbors for point", i, timeMod.time() - startTime
        neighborDistances, neighborIndexes = kdTree.query(pointData[i], numNeighbors)
        neighbors = [data[ind] for ind in neighborIndexes]

        sampleClass = drumClasses[tuple(data[i])]
        neighborClasses = [drumClasses[tuple(neighbor)] for neighbor in neighbors]
        numCorrectNeighborClasses = len(filter(lambda c : c == sampleClass, neighborClasses))
        fracCorrectNeighbors[i] = numCorrectNeighborClasses * 1.0 / numNeighbors
        
    return fracCorrectNeighbors
    #from here do we want - mean, median, mode, stddev of correctClass fraction?
    #mean, median, mode, stddev of correctClass frac per class
    #% accuracy over a thresold?

def calculateFuncPerClass(data, numPerClass, calcFunc):
    segments = [0]+numPerClass
    for i in range(1, len(segments)):
        segments[i] = segments[i] + segments[i-1]
    valuesPerClass = []
    for i in range(len(segments)-1):
        valuesPerClass.append(calcFunc(data[segments[i]:segments[i+1]]))
    return valuesPerClass

numNeighbors = 10




In [116]:
classAccuracies_HD = calculateKNNClassAccuracy(data, numNeighbors, True, kdTree1024Dim)

building tree 1507407928.65
tree built 1507407928.65
calculating nearest neighbor classes
evaluating neighbors for point 0 0.000815868377686
evaluating neighbors for point 1000 115.909148932
evaluating neighbors for point 2000 238.807391882
evaluating neighbors for point 3000 356.995471954
evaluating neighbors for point 4000 474.88192296
evaluating neighbors for point 5000 589.373081923
evaluating neighbors for point 6000 718.804381847
evaluating neighbors for point 7000 855.206861019
evaluating neighbors for point 8000 990.591647863
evaluating neighbors for point 9000 1125.77812099
evaluating neighbors for point 10000 1270.26736403


In [127]:
classAccuracies_LD = calculateKNNClassAccuracy(newData, numNeighbors, True, kdTree2Dim)

building tree 1507410395.51
tree built 1507410395.51
calculating nearest neighbor classes
evaluating neighbors for point 0 0.000693798065186
evaluating neighbors for point 1000 1.13937783241
evaluating neighbors for point 2000 2.22474694252
evaluating neighbors for point 3000 3.29074978828
evaluating neighbors for point 4000 4.38694787025
evaluating neighbors for point 5000 5.47066187859
evaluating neighbors for point 6000 6.56675577164
evaluating neighbors for point 7000 7.62933897972
evaluating neighbors for point 8000 8.69908380508
evaluating neighbors for point 9000 9.81199383736
evaluating neighbors for point 10000 10.8873569965


In [125]:
len(classAccuracies_HD)

10560

In [126]:
import cPickle
cPickle.dump(classAccuracies_HD, open("1024DimClassAccuracies10neighbors.np", "w"))

In [ ]:
import numpy as np
y = np.zeros((2, 3, 4))
for i in range(2):
    for j in range(3):
        for k in range(4):
            y[i][j][k] = str(i*12 + j*4 + k)
y.shape = (2, 12)
a = {}
a[tuple(y[0])] = 5

In [ ]:
y.shape = (24)
print y[0:10]

In [ ]:
from multiprocessing import Pool
pool = Pool()
print pool.map(str, [1, 2, 3])

In [ ]:
# use this to calculate "k-nearest-neighbors class-preservation" metric
# https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html

In [ ]:
a = [1, 2, 3, 4]
a.partition(0,2,4)